In [1]:
import pandas as pd
import numpy as np
import os

# loading in the training and test datasets
train_data = pd.read_csv('dmt-2025-2nd-assignment/training_set_VU_DM.csv')
test_data = pd.read_csv('dmt-2025-2nd-assignment/test_set_VU_DM.csv')

In [2]:
# The assignment said there are nearly 5 million observations, verifying
print(train_data.shape) # (4958347, 54)
print(test_data.shape)  # (4959183, 50)

# The training data contains four columns that are not in the testing data. I will look at observations only where a booking was made
print(train_data[train_data['gross_bookings_usd'].notna()][['position', 'click_bool', 'booking_bool', 'gross_bookings_usd']].head(10))

# What about observations where no booking was made
print(train_data[train_data['gross_bookings_usd'].isna()][['position', 'click_bool', 'booking_bool', 'gross_bookings_usd']].head(10))

(4958347, 54)
(4959183, 50)
     position  click_bool  booking_bool  gross_bookings_usd
12         13           1             1              114.29
63          1           1             1              162.38
68         16           1             1               96.41
194         4           1             1              222.58
211        22           1             1               47.10
243        26           1             1              286.32
274         1           1             1             1927.64
312        13           1             1             1831.30
346         4           1             1              226.76
387         1           1             1               60.77
   position  click_bool  booking_bool  gross_bookings_usd
0        27           0             0                 NaN
1        26           0             0                 NaN
2        21           0             0                 NaN
3        34           0             0                 NaN
4         4           

In [3]:
# One of the columns is the logged historical price over the last trading period, where a 0 is if it wasn't sold then. I will convert this to standard USD and create a column comparing that to the given price. However the price may be per night or for the whole stay, depending on the country. The srch_length_of_stay will be used to determine that.
train_data['prop_historical_price'] = train_data['prop_log_historical_price'].replace(0, np.nan) # no meaningful comparison can be done
train_data['prop_historical_price'] = np.exp(train_data['prop_historical_price']).round(2) # converting to USD

# In order to determine if the price is per night or the whole stay, I will divide the displayed price with the number of nights the user specified. This will be compared to the historical price in USD. If the value is significantly lower than the historical price, then the price is per night. If it is around the same, then the price is for the whole stay.
train_data['price_usd_per_night_test'] = train_data['price_usd'] / train_data['srch_length_of_stay']

# display adjustments
pd.set_option('display.max_columns', None)   # show all columns
pd.set_option('display.width', 1000)         # or set to a large number
pd.set_option('display.max_colwidth', None)  # avoid truncating long cell contents

# One of the variables is whether a hotel has a sales promotion currently displayed. That would affect the current price but not the historical price, and it would mean the current price is lower than it historically should be. I will use 25% for the adjustment, which is only for observations where the hotel has a sale

# Decreasing a price value by 25% means the decreased value has to be increased by 33% in order to get back to what it originally was
train_data['price_usd_without_promo'] = np.where(train_data['promotion_flag'] == 1, train_data['price_usd'] * 1.33, train_data['price_usd']).round(2)

# actually I think the historical_price column is already per night, or at least the description doesn't say anything about that. I will calculate a ratio between the two prices and see if it is significantly different than 1
train_data['price_ratio'] = train_data['price_usd_without_promo'] / train_data['prop_historical_price']

# looking at the ratio
print(train_data[['prop_historical_price', 'price_usd_without_promo', 'srch_length_of_stay', 'prop_country_id', 'price_ratio', 'promotion_flag']].head(50))

# displaying quantile statistics of the ratio
print(train_data['price_ratio'].describe())

# looking at the smallest values
print(train_data[train_data['price_ratio'] < 0.35][['prop_historical_price', 'price_usd_without_promo', 'srch_length_of_stay', 'prop_country_id', 'price_ratio', 'promotion_flag']].head(50))

# The adjusted USD price without promotion is significantly lower than the historical price. I will assume that means the price is per night. Checking out the other end of the spectrum
print(train_data[train_data['price_ratio'] > 1.5][['prop_historical_price', 'price_usd_without_promo', 'srch_length_of_stay', 'prop_country_id', 'price_ratio', 'promotion_flag']].head(50))

# For cases where the ratio is significantly higher than the historical price, I will assume that means the price is for the whole stay. It will have to be adjusted to be per night.

# I will group the data by country and compute the percentage of hotels that have a low price ratio & high price ratio
low_ratio = train_data[train_data['price_ratio'] < 0.35].groupby('prop_country_id').size() / train_data.groupby('prop_country_id').size()
low_ratio.fillna(0, inplace=True)

# only keeping countries that have a high proportion of low ratios
low_ratio = low_ratio[low_ratio > 0.5]
print(low_ratio) # 1 observation

# That is good there are almost no observations, as I already said they are per night. Verifying the high ratio countries
high_ratio = train_data[train_data['price_ratio'] > 1.5].groupby('prop_country_id').size() / train_data.groupby('prop_country_id').size()
high_ratio.fillna(0, inplace=True)

# only keeping countries that have a high proportion of high ratios
high_ratio = high_ratio[high_ratio > 0.4]
print(high_ratio) # 4, only these countries have high ratios

# Print observations for these 4 countries
print(train_data[train_data['prop_country_id'].isin(high_ratio.index)][['prop_historical_price', 'price_usd_without_promo', 'srch_length_of_stay', 'prop_country_id', 'price_ratio', 'promotion_flag']].head(50))

# adjusting the values for the countries with a high ratio
train_data.loc[train_data['prop_country_id'].isin(high_ratio.index), 'price_usd_without_promo'] = train_data['price_usd_without_promo'] / train_data['srch_length_of_stay']


    prop_historical_price  price_usd_without_promo  srch_length_of_stay  prop_country_id  price_ratio  promotion_flag
0                  141.17                   104.77                    1              219     0.742155               0
1                  152.93                   170.74                    1              219     1.116459               0
2                  137.00                   179.80                    1              219     1.312409               0
3                   80.64                   602.77                    1              219     7.474826               0
4                  138.38                   143.58                    1              219     1.037578               0
5                  181.27                   195.32                    1              219     1.077509               0
6                  122.73                   129.35                    1              219     1.053940               0
7                   62.80                    85.37      

In [4]:
# What is the average price per night for countries? Computing a weighted average between current price and nights
weighted_avg = (
    train_data
    .assign(weighted_price=lambda df: df['price_usd_without_promo'] * df['srch_length_of_stay'])
    .groupby('prop_country_id')
    .agg(
        total_weighted_price=('weighted_price', 'sum'),
        total_stay=('srch_length_of_stay', 'sum')
    )
    .assign(weighted_avg_price_usd_without_promo=lambda df: df['total_weighted_price'] / df['total_stay'])
    .round(2)
    .reset_index()[['prop_country_id', 'weighted_avg_price_usd_without_promo']]
)

print(weighted_avg)

# quantile statistics
print(weighted_avg['weighted_avg_price_usd_without_promo'].describe())

# looking at the most expensive countries
print(weighted_avg[weighted_avg['weighted_avg_price_usd_without_promo'] > 1000][['prop_country_id', 'weighted_avg_price_usd_without_promo']])

     prop_country_id  weighted_avg_price_usd_without_promo
0                  1                                223.43
1                  2                                137.58
2                  4                                149.63
3                  7                                132.99
4                  9                                131.30
..               ...                                   ...
167              224                                 87.92
168              225                                387.06
169              226                                166.79
170              229                                101.32
171              230                                246.43

[172 rows x 2 columns]
count     172.000000
mean      309.350000
std       610.837121
min        32.740000
25%       137.302500
50%       183.685000
75%       260.750000
max      5426.590000
Name: weighted_avg_price_usd_without_promo, dtype: float64
     prop_country_id  weighted_avg_price_

In [5]:
train_data[train_data['prop_country_id'].isin([53, 39, 34])][['prop_historical_price', 'price_usd_without_promo', 'srch_length_of_stay', 'prop_country_id', 'price_ratio', 'promotion_flag']]. \
    sort_values(by=['price_usd_without_promo', 'prop_country_id'], ascending = False)

,prop_historical_price,price_usd_without_promo,srch_length_of_stay,prop_country_id,price_ratio,promotion_flag
1168566,NaN,19726328.00,4,39,NaN,0
3117007,160.77,9381308.71,1,53,58352.358711,0
1168574,NaN,5444467.00,4,39,NaN,0
1168581,NaN,5194731.29,4,39,NaN,1
1168576,NaN,4884239.00,4,39,NaN,0
...,...,...,...,...,...,...
3402025,86.49,0.01,1,39,0.000116,0
3402026,68.03,0.01,1,39,0.000147,1
3402029,74.44,0.01,1,39,0.000134,0
3402030,75.94,0.01,1,39,0.000132,0


In [6]:
# A lot of these values are extremely high and unrealistic. I will make adjustments to observations that have a price higher than 1000. For ones that have the historical value, I will use that instead. Otherwise for NaNs I do current price / nights

# This is needed for the where condition so that only these rows are affected by the calculation
mask = train_data['price_usd_without_promo'] > 1000

# doing the calculation
train_data.loc[mask, 'price_usd_without_promo'] = np.where(
    train_data.loc[mask, 'prop_historical_price'].notna(),
    train_data.loc[mask, 'prop_historical_price'],
    train_data.loc[mask, 'price_usd_without_promo'] / train_data.loc[mask, 'srch_length_of_stay']
)

In [7]:
# What is the average price per night for countries? Computing a weighted average between current price and nights
weighted_avg = (
    train_data
    .assign(weighted_price=lambda df: df['price_usd_without_promo'] * df['srch_length_of_stay'])
    .groupby('prop_country_id')
    .agg(
        total_weighted_price=('weighted_price', 'sum'),
        total_stay=('srch_length_of_stay', 'sum')
    )
    .assign(weighted_avg_price_usd_without_promo=lambda df: df['total_weighted_price'] / df['total_stay'])
    .round(2)
    .reset_index()[['prop_country_id', 'weighted_avg_price_usd_without_promo']]
)

print(weighted_avg)

# quantile statistics
print(weighted_avg['weighted_avg_price_usd_without_promo'].describe()) # much much better than before

     prop_country_id  weighted_avg_price_usd_without_promo
0                  1                                223.43
1                  2                                137.58
2                  4                                149.53
3                  7                                132.99
4                  9                                130.96
..               ...                                   ...
167              224                                 85.82
168              225                                150.34
169              226                                166.79
170              229                                101.32
171              230                                241.12

[172 rows x 2 columns]
count     172.000000
mean      197.838140
std       129.457518
min        32.740000
25%       132.937500
50%       167.475000
75%       225.297500
max      1468.740000
Name: weighted_avg_price_usd_without_promo, dtype: float64


In [8]:
# Do any hotels have NaN price values?
print(train_data[train_data['price_usd_without_promo'].isna()][['prop_historical_price', 'price_usd_without_promo', 'srch_length_of_stay', 'prop_country_id', 'price_ratio', 'promotion_flag']].head(50)) # empty dataframe

Empty DataFrame
Columns: [prop_historical_price, price_usd_without_promo, srch_length_of_stay, prop_country_id, price_ratio, promotion_flag]
Index: []
